# Decorators

In general decorator function:
* takes function as argument
* returns closure
* runs inner function (closure)
* returns what the inner function returns given the provided parameters

look again at "count function" closure example from last lesson, executed function that was passed in and also counted how many times that passed in function was called. What the count function does in addition to the mere function execution is what makes it a decorator function. 
Example from before:

In [1]:
def counter(fn): #fn is local to counter, but since used in inner, it is nonlocal
    
    cnt = 0  # initially fn has been run zero times
    
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt)) # "decoration"
        return fn(*args, **kwargs)
    
    return inner

In [10]:
def add(a, b=0):    
    return "result:", a + b


In [11]:
counted_add = counter(add)
counted_add(1, 2)

add has been called 1 times


('result:', 3)

### common way to define decorators

In [12]:
add=counter(add)

In [13]:
add(5,10)

add has been called 1 times


('result:', 15)

### using @ for decorators:

with "@" function is defined and decorated directly:

In [14]:
@counter
def mult(a,b=1):
    return a*b

In [15]:
mult(2,2)

mult has been called 1 times


4

In [17]:
# same as:
def mult(a,b=1):
    return a*b
mult=counter(mult)

--> "@" is for convenience, since it is so common to specify a decorator like above

In [18]:
@counter
def mult(a,b=1):
    return a*b

In [19]:
mult.__name__

'inner'

inner because mult is passed as argument into counter where it is used in inner function/ closure. Hence becomes "inner"

In [20]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



unfortunate for debugging since all functions used in counter get the "inner" name

In [21]:
#solution:

def counter(fn):
    cnt = 0  
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    inner.__name__=fn.__name__    # <--
    inner.__doc__=fn.__doc__      # <--
    
    return inner

In [22]:
@counter
def mult(a,b=1):
    return a*b

In [23]:
mult.__name__

'mult'

In [24]:
# not fixed:
help(mult)

Help on function mult in module __main__:

mult(*args, **kwargs)



#### functools wraps function

fixes metadata of decorated function better than above fix

In [22]:
from functools import wraps

wraps is a decorator itself, but also takes in parameter next to decorated function

In [23]:
# called like this:
def counter(fn):
    cnt = 0  
    
    @wraps(fn)        # decorates "inner" with information from fn, i.e. the passed function
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt = cnt + 1
        print('{0} has been called {1} times'.format(fn.__name__, cnt))
        return fn(*args, **kwargs)
    
    return inner

In [24]:
@counter
def divide(a : int ,b : int =1):
    return b and a/b

In [25]:
divide(12,3)

divide has been called 1 times


4.0

In [26]:
divide(10,0)

divide has been called 2 times


0

In [27]:
divide.__name__

'divide'

In [28]:
help(divide)

Help on function divide in module __main__:

divide(a: int, b: int = 1)



In [29]:
import inspect
inspect.signature(divide)

<Signature (a: int, b: int = 1)>

#### own example:

In [5]:
# own example:
import numpy
# closure:
def add_error(fn):
    '''adds a random error term between 1 and 10 to result of a calculation'''
    err=numpy.random.randint(0,10)
    def inner(*args,**kwargs):
        return fn(*args,**kwargs)+ err
    
    return inner

In [6]:
@add_error
def addi(a,b=0):
    return a+b

In [9]:
addi(1)

2

In [10]:
addi(3)

4

--> since err variable outside of inner function, it is set when add_error function is called and then remains the same. Here with always warrying error:

In [11]:

import numpy
# closure:
def add_error(fn):
    '''adds a random error term between 1 and 10 to result of a calculation'''
    def inner(*args,**kwargs):
        err=numpy.random.randint(0,10)
        return fn(*args,**kwargs)+ err
    
    return inner

In [12]:
@add_error
def addi(a,b=0):
    return a+b

In [16]:
addi(1),addi(1+3), addi(3+4)

(4, 4, 7)

In [17]:
addi.__name__

'inner'

In [30]:
# now with proper naming:
import numpy
from functools import wraps
# closure:
def add_error(fn):
    '''adds a random error term between 1 and 10 to result of a calculation'''
    @wraps(fn)
    def inner(*args,**kwargs):
        err=numpy.random.randint(0,10)
        return fn(*args,**kwargs)+ err
    
    return inner

In [34]:
@add_error
def multi(a,b=1):
    '''multiplies a with b'''
    return a*b

In [35]:
multi(2,5), multi(1), multi(10,2)

(14, 4, 21)

In [36]:
multi.__name__

'multi'

In [37]:
help(multi)

Help on function multi in module __main__:

multi(a, b=1)
    multiplies a with b

